In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
import pandas as pd
import torch

In [17]:
train_df = pd.read_csv("train.csv")  # يحتوي على input, output
val_df = pd.read_csv("val.csv")

In [18]:
train_df = train_df[["input", "output"]]
val_df = val_df[["input", "output"]]

In [19]:
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

In [20]:
model_name = "UBC-NLP/AraT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

loading configuration file config.json from cache at C:\Users\DELL\.cache\huggingface\hub\models--UBC-NLP--AraT5-base\snapshots\b99444fa197df5a598e0a426e76a31e4a43d1d23\config.json
Model config T5Config {
  "_name_or_path": "UBC-NLP/AraT5-base",
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.43.4",
  "use_cache": true,
  "vocab_size": 110080
}

loading file spiece.model from cache at C:\Users\DELL\.cache\huggingface\hub\models--UBC

In [21]:
def tokenize(batch):
    model_inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

Map:   0%|          | 0/727623 [00:00<?, ? examples/s]

C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/80848 [00:00<?, ? examples/s]

In [23]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),  # لتسريع التدريب
    report_to="none"
)

C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices


In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

Using auto half precision backend
Exception ignored in: <function tqdm.__del__ at 0x00000212CE965C10>
Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
KeyboardInterrupt: 
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: input, output. If input, output are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 727,623
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 45,476
  Number of trainable parameters = 282,770,688


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("./models/arabic_t5_medical", safe_serialization=False)
tokenizer.save_pretrained("./models/arabic_t5_medical")